In [25]:
import pandas as pd
import requests
import folium
from IPython.display import IFrame

In [26]:
country_geo = 'world-countries.json'
re = requests.get("https://corona.lmao.ninja/v2/countries?yesterday&sort")
re = re.json()
re = pd.DataFrame.from_dict(re)
pd.set_option('display.max_columns', None)
country_info = re['countryInfo']
country_info = pd.json_normalize(country_info)
df = pd.concat([re, country_info], axis=1,sort=False)
df = df.drop('countryInfo', axis=1)

In [27]:
def thousand_separator(val):
    return f"{val:,}"

df['recovered'] = df['recovered'].apply(thousand_separator)
df['cases'] = df['cases'].apply(thousand_separator)
df['deaths'] = df['deaths'].apply(thousand_separator)


In [28]:
m = folium.Map(
    location=[100, 0],
    zoom_start=1.5,
)

In [29]:
folium.Choropleth (
    geo_data= country_geo,
    data=df,
    columns=['lat', 'long'],
    key_on='feature.id',
    fill_color='YlGnBu',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Covid Analysis'
).add_to(m)

In [30]:
def circle_maker(x):
    folium.Circle(location=[x[0],x[1]],
                 radius=10,
                 color="red",
                 popup='{}\n Cases: {}\n Deaths: {}\n Recovered: {}'.format(x[2], x[3], x[4], x[5])).add_to(m)
df[['lat','long','country','cases','deaths', 'recovered']].apply(lambda x:circle_maker(x),axis=1)
m.save('covid_data.html')

In [31]:
IFrame(src=m, width=450, height=450)
folium.Popup(src=m, max_width=1000)